In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from scipy.stats import skewnorm, norm
import arviz as az

In [3]:
rng = np.random.default_rng(10)
x = skewnorm(3, loc=5, scale=2).rvs(size=50, random_state=rng)
y = - np.ones_like(x)

## Computing ecdf animation

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from functools import partial

fig, ax = plt.subplots()
rugs = [ax.plot([], [], "|k", markersize=10)[0] for _ in x]
ecdf, = ax.plot([], [], "b-", linewidth=2, drawstyle="steps-post")

ax.legend(
    handles=[ecdf, rugs[0]],
    labels=["ECDF", "Dades"],
    loc="upper left",
)

def init():
    ax.set_title('Funció de distribució empírica (ECDF)')
    #ax.spines.left.set_color('none')
    ax.spines.right.set_color('none')
    ax.spines.bottom.set_position('zero')
    ax.spines.top.set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    #ax.tick_params(left=False, labelleft=False)
    ax.set_ylim(-0.1, 1)
    ax.set_xlim(x.min()-0.1, x.max()+0.1)
    ecdf.set_data([], [])
    for r in rugs:
        r.set_data([], [])
    return tuple([ecdf, *rugs])

def update(frame, samples, sorted_samples):
    step, idx = frame
    if step == "rug":
        for rug, sample in zip(rugs, samples):
            rug.set_data([sample], [-0.05])
            rug.set_color("black")
    elif step == "sort":
        for rug, sample in zip(rugs, sorted_samples[1:]):
            rug.set_data([sample], [-0.05])
    elif step == "ecdf":
        rugs[idx].set(color="red", markersize=30)
        if idx > 0:
            rugs[idx-1].set(color="lightgray", markersize=10)
        ecdf.set_data(sorted_samples[:idx+2], np.linspace(0,  (idx+1) / len(samples), idx+2))
    elif step == "cleanup":
        rugs[-1].set(color="lightgray", markersize=10)
    return tuple([ecdf, *rugs])

frames = [
    *[("wait", None)]*2,
    ("rug", None),
    *[("wait", None)]*2,
    ("sort", None),
    *[("wait", None)]*2,
    *[("ecdf", i) for i in range(len(x))],
    ("cleanup", None),
    *[("wait", None)]*6,
]

sorted_x = np.sort(x)
ani = FuncAnimation(
    fig, partial(update, samples=x, sorted_samples=np.hstack([sorted_x[0], sorted_x])),
    frames=frames,
    init_func=init,
    blit=True,
    interval=300,
)

#ani.save("ecdf.ca.mp4", dpi=300)

with open("ecdf.ca.html", mode="w", encoding="utf-8") as f:
    f.write(ani.to_jshtml())

plt.show()

##